# **Sales Data Imputation System**

# **Milestone 1**

# **Recommendation System Imputation: Sales Data Example**

--------------
## **Context**
--------------

Data integity issues in Point-of-Sale (POS) data continues to be an issue for large companies, providing opportunity for advanced imputation methods like leveraging recommendation systems.

----------------
## **Objective**
----------------

Build a recommendation system to recommend (impute) sales for customers entirely or partially without POS data.

-----------------------------
## **Dataset** 
-----------------------------

The generic sales_data dataset contains the following attributes:

- **Order Number:** Every order identified with a unique order-line id
- **Order Line Number:** Every order identified with a unique order-line id
- **Price Per Unit:** The rating of the corresponding product by the corresponding user
- **Order Date:** Time of the rating. We **will not use this column** to solve the current problem
- **Status:** Order status of the product.

### **Importing Libraries and the Dataset**

In [1]:
#Mounting the drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np 
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns 

from sklearn.metrics.pairwise import cosine_similarity 
from collections import defaultdict

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

In [29]:
#importing the datasets
pos_df = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/sales_data.csv', encoding= 'unicode_escape')
bookings_df = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/bookings_data.csv')

### **Reviewing the Data**

In [30]:
# See top 10 records of pos_df data
pos_df.head(10)

,ORDERNUMBER,ORDERLINENUMBER,PRICEEACH,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,MSRP,...,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE,QUANTITYORDERED,SALES
0,10134,2,94.74,7/1/2003 0:00,Shipped,3,7,2003,Motorcycles,95,...,Paris,NaN,75508,France,EMEA,Da Cunha,Daniel,Medium,41.0,3884.34
1,10145,6,83.26,8/25/2003 0:00,Shipped,3,8,2003,Motorcycles,95,...,Pasadena,CA,90003,USA,NaN,Young,Julie,Medium,45.0,3746.70
2,10159,14,100.00,10/10/2003 0:00,Shipped,4,10,2003,Motorcycles,95,...,San Francisco,CA,NaN,USA,NaN,Brown,Julie,Medium,49.0,5205.27
3,10168,1,96.66,10/28/2003 0:00,Shipped,4,10,2003,Motorcycles,95,...,Burlingame,CA,94217,USA,NaN,Hirano,Juri,Medium,36.0,3479.76
4,10180,9,86.13,11/11/2003 0:00,Shipped,4,11,2003,Motorcycles,95,...,Lille,NaN,59000,France,EMEA,Rance,Martine,Small,29.0,2497.77
5,10188,1,100.00,11/18/2003 0:00,Shipped,4,11,2003,Motorcycles,95,...,Bergen,NaN,N 5804,Norway,EMEA,Oeztan,Veysel,Medium,48.0,5512.32
6,10201,2,98.57,12/1/2003 0:00,Shipped,4,12,2003,Motorcycles,95,...,San Francisco,CA,NaN,USA,NaN,Murphy,Julie,Small,22.0,2168.54
7,10211,14,100.00,1/15/2004 0:00,Shipped,1,1,2004,Motorcycles,95,...,Paris,NaN,75016,France,EMEA,Perrier,Dominique,Medium,41.0,4708.44
8,10223,1,100.00,2/20/2004 0:00,Shipped,1,2,2004,Motorcycles,95,...,Melbourne,Victoria,3004,Australia,APAC,Ferguson,Peter,Medium,37.0,3965.66
9,10237,7,100.00,4/5/2004 0:00,Shipped,2,4,2004,Motorcycles,95,...,NYC,NY,10022,USA,NaN,Frick,Michael,Small,23.0,2333.12


In [31]:
# See top 10 records of bookings_df data
bookings_df.head(10)

,BOOKINGSORDERDATE,BOOKINGSSTATUS,PRODUCTCODE,CUSTOMERNAME,BOOKINGSCOUNTRY,BOOKINGSTERRITORY,BOOKINGSQUANTITYORDERED
0,7/1/2003 0:00,Shipped,S10_1678,Lyon Souveniers,France,EMEA,7768.0
1,8/25/2003 0:00,Shipped,S10_1678,Toys4GrownUps.com,USA,NaN,7492.0
2,10/10/2003 0:00,Shipped,S10_1678,Corporate Gift Ideas Co.,USA,NaN,10410.0
3,10/28/2003 0:00,Shipped,S10_1678,Technics Stores Inc.,USA,NaN,6958.0
4,11/11/2003 0:00,Shipped,S10_1678,Daedalus Designs Imports,France,EMEA,4994.0
5,11/18/2003 0:00,Shipped,S10_1678,Herkku Gifts,Norway,EMEA,11024.0
6,12/1/2003 0:00,Shipped,S10_1678,Mini Wheels Co.,USA,NaN,4336.0
7,1/15/2004 0:00,Shipped,S10_1678,Auto Canal Petit,France,EMEA,9416.0
8,2/20/2004 0:00,Shipped,S10_1678,"Australian Collectors, Co.",Australia,APAC,7930.0
9,4/5/2004 0:00,Shipped,S10_1678,Vitachrome Inc.,USA,NaN,4666.0


### **Info Types + Description**

In [32]:
# See the info of the bookings_df data
pos_df.info()
pos_df.describe().T

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2836 entries, 0 to 2835
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ORDERNUMBER       2836 non-null   int64  
 1   ORDERLINENUMBER   2836 non-null   int64  
 2   PRICEEACH         2817 non-null   float64
 3   ORDERDATE         2836 non-null   object 
 4   STATUS            2836 non-null   object 
 5   QTR_ID            2836 non-null   int64  
 6   MONTH_ID          2836 non-null   int64  
 7   YEAR_ID           2836 non-null   int64  
 8   PRODUCTLINE       2836 non-null   object 
 9   MSRP              2836 non-null   int64  
 10  PRODUCTCODE       2836 non-null   object 
 11  CUSTOMERNAME      2836 non-null   object 
 12  PHONE             2836 non-null   object 
 13  ADDRESSLINE1      2836 non-null   object 
 14  ADDRESSLINE2      305 non-null    object 
 15  CITY              2836 non-null   object 
 16  STATE             1343 non-null   object 


,count,mean,std,min,25%,50%,75%,max
ORDERNUMBER,2836.0,10261.253879,96.661615,10100.00,10180.00,10263.00,10336.0,10788.0
ORDERLINENUMBER,2836.0,6.467207,4.229732,1.00,3.00,6.00,9.0,18.0
PRICEEACH,2817.0,83.631885,20.185024,26.88,68.78,95.55,100.0,100.0
QTR_ID,2836.0,2.721086,1.202933,1.00,2.00,3.00,4.0,4.0
MONTH_ID,2836.0,7.101904,3.654359,1.00,4.00,8.00,11.0,12.0
YEAR_ID,2836.0,2003.814528,0.698813,2003.00,2003.00,2004.00,2004.0,2005.0
MSRP,2836.0,100.605783,40.017339,33.00,68.00,99.00,124.0,214.0
QUANTITYORDERED,2817.0,35.104011,9.743597,6.00,27.00,35.00,43.0,97.0
SALES,2817.0,3552.627497,1842.489205,482.13,2202.48,3184.80,4508.0,14082.8


In [33]:
# See the info of the pos_df data
pos_df.info()
pos_df.describe().T
# print(pos_df.nunique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2836 entries, 0 to 2835
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ORDERNUMBER       2836 non-null   int64  
 1   ORDERLINENUMBER   2836 non-null   int64  
 2   PRICEEACH         2817 non-null   float64
 3   ORDERDATE         2836 non-null   object 
 4   STATUS            2836 non-null   object 
 5   QTR_ID            2836 non-null   int64  
 6   MONTH_ID          2836 non-null   int64  
 7   YEAR_ID           2836 non-null   int64  
 8   PRODUCTLINE       2836 non-null   object 
 9   MSRP              2836 non-null   int64  
 10  PRODUCTCODE       2836 non-null   object 
 11  CUSTOMERNAME      2836 non-null   object 
 12  PHONE             2836 non-null   object 
 13  ADDRESSLINE1      2836 non-null   object 
 14  ADDRESSLINE2      305 non-null    object 
 15  CITY              2836 non-null   object 
 16  STATE             1343 non-null   object 


,count,mean,std,min,25%,50%,75%,max
ORDERNUMBER,2836.0,10261.253879,96.661615,10100.00,10180.00,10263.00,10336.0,10788.0
ORDERLINENUMBER,2836.0,6.467207,4.229732,1.00,3.00,6.00,9.0,18.0
PRICEEACH,2817.0,83.631885,20.185024,26.88,68.78,95.55,100.0,100.0
QTR_ID,2836.0,2.721086,1.202933,1.00,2.00,3.00,4.0,4.0
MONTH_ID,2836.0,7.101904,3.654359,1.00,4.00,8.00,11.0,12.0
YEAR_ID,2836.0,2003.814528,0.698813,2003.00,2003.00,2004.00,2004.0,2005.0
MSRP,2836.0,100.605783,40.017339,33.00,68.00,99.00,124.0,214.0
QUANTITYORDERED,2817.0,35.104011,9.743597,6.00,27.00,35.00,43.0,97.0
SALES,2817.0,3552.627497,1842.489205,482.13,2202.48,3184.80,4508.0,14082.8


In [34]:
# Left merge the bookings_df and pos_df data on 'PRODUCTCODE' and 'CUSTOMERNAME'.
df = pd.merge(bookings_df, pos_df, how='left', on=['PRODUCTCODE', 'CUSTOMERNAME'])

In [35]:
df.head()

,BOOKINGSORDERDATE,BOOKINGSSTATUS,PRODUCTCODE,CUSTOMERNAME,BOOKINGSCOUNTRY,BOOKINGSTERRITORY,BOOKINGSQUANTITYORDERED,ORDERNUMBER,ORDERLINENUMBER,PRICEEACH,...,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE,QUANTITYORDERED,SALES
0,7/1/2003 0:00,Shipped,S10_1678,Lyon Souveniers,France,EMEA,7768.0,10134,2,94.74,...,Paris,NaN,75508,France,EMEA,Da Cunha,Daniel,Medium,41.0,3884.34
1,7/1/2003 0:00,Shipped,S10_1678,Lyon Souveniers,France,EMEA,7768.0,10465,2,NaN,...,Paris,NaN,75508,France,EMEA,Da Cunha,Daniel,Medium,NaN,NaN
2,8/25/2003 0:00,Shipped,S10_1678,Toys4GrownUps.com,USA,NaN,7492.0,10145,6,83.26,...,Pasadena,CA,90003,USA,NaN,Young,Julie,Medium,45.0,3746.70
3,8/25/2003 0:00,Shipped,S10_1678,Toys4GrownUps.com,USA,NaN,7492.0,10482,6,NaN,...,Pasadena,CA,90003,USA,NaN,Young,Julie,Medium,NaN,NaN
4,10/10/2003 0:00,Shipped,S10_1678,Corporate Gift Ideas Co.,USA,NaN,10410.0,10159,14,100.00,...,San Francisco,CA,NaN,USA,NaN,Brown,Julie,Medium,49.0,5205.27
